# Import Libraries

In [ ]:
# Import PyTorch for tensor operations and building neural networks
import torch
# nn module provides essential tools for building neural networks
import torch.nn as nn
# optim module contains optimizers to train the model
import torch.optim as optim
# Datasets and pre-trained models from torchvision
from torchvision import datasets, transforms, models
# DataLoader is used to efficiently load data in batches during training
from torch.utils.data import Dataset, DataLoader, random_split
# For learning rate scheduling
from torch.optim.lr_scheduler import StepLR
# Display the model architecture summary
from torchsummary import summary
# tqdm is used to display progress bars during training or processes
from tqdm import tqdm
# numpy is used for numerical computations, such as arrays and matrices
import numpy as np
# pandas is used for data manipulation and analysis, often used with DataFrames
import pandas as pd
# matplotlib is used for plotting graphs and visualizing data
import matplotlib.pyplot as plt
# Used to measure training time or process durations
import time
# Import the random module to enable random number generation for various operations like shuffling or sampling
import random
# Import the os module for interacting with the operating system, including file system operations and environment variable access
import os
# interact with the Python runtime environment, including input/output redirection
import sys
# Python Imaging Library (PIL) to handle image loading and manipulation
from PIL import Image
# Importing required types from the typing module
from typing import Tuple, List, Dict

# Import necessary libraries for resource monitoring
import psutil  # For CPU and Memory usage monitoring
import gc  # For garbage collection
from pynvml import *  # NVIDIA GPU monitoring library

torch.cuda.empty_cache()  # Clear unused memory

# Initialize the NVIDIA Management Library (NVML)
nvmlInit()  # This initializes the NVML library for GPU monitoring

#### Redirecting Print Output to a Log File

In [ ]:
# Define the main parameters
MODEL = "AlexNet" # Target Model
DATASET = "EMNIST" # Target Dataset
NUM_CLASSES = 26 # No of classes
ALPHA = 0 # Couplin Factor

# Define the folder to save the Results
FOLDER_NAME = f"{MODEL}-{DATASET}"
# Check if the folder exists, if not create it
if not os.path.exists(FOLDER_NAME):
    os.makedirs(FOLDER_NAME)

# Save the log file in the specified folder
log_file_path = os.path.join(FOLDER_NAME, f"{MODEL}_{DATASET}_a{ALPHA}-logfile.txt")
log_file = open(log_file_path, 'w')

# Redirect the standard output to the file
sys.stdout = log_file

## Reproducibility

In [ ]:
# Define seed value
seed = 42

# Set the random seed for reproducibility
random.seed(seed)  # Python's random module
np.random.seed(seed)  # NumPy random module
torch.manual_seed(seed)  # PyTorch CPU random seed
# Check if CUDA is available and set the seed for CUDA as well
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)  # PyTorch GPU random seed (for current device)
    torch.cuda.manual_seed_all(seed) # PyTorch GPU random seed (for all devices, if multi-GPU)

# For deterministic behavior with cuDNN (when using GPU)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False  # Disables the cudnn autotuner to ensure reproducibility

# Check the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'{device} is available...')
# Get the number of GPUs (if any) available using PyTorch
num_gpus = torch.cuda.device_count()
print(f"Number of GPUs available: {num_gpus}")
# If GPUs are available, print the name of the first GPU
if num_gpus > 0:
    print(f"GPU Name: {torch.cuda.get_device_name(0)}\n")  # Print the name of the first GPU

## GPU & CPU monitoring

In [ ]:
def summarize_gpu_info():
    # Get the number of GPUs using PyTorch's CUDA device count
    num_gpus = torch.cuda.device_count()

    # Loop through each GPU (from 0 to num_gpus - 1)
    for i in range(num_gpus):
        handle = nvmlDeviceGetHandleByIndex(i)  # Get GPU device handle for the current GPU
        # Retrieve memory information from the GPU
        mem_info = nvmlDeviceGetMemoryInfo(handle)
        # Retrieve GPU utilization information
        gpu_util = nvmlDeviceGetUtilizationRates(handle)
        # Retrieve GPU temperature information
        gpu_temp = nvmlDeviceGetTemperature(handle, NVML_TEMPERATURE_GPU)

        # Display GPU memory usage, utilization, and temperature for each GPU
        print(f"GPU {i}:")
        print(f"  Memory Usage: {mem_info.used / 1024 ** 2} MB (Used) / {mem_info.total / 1024 ** 2} MB (Total)")
        print(f"  GPU Utilization: {gpu_util.gpu} %")
        print(f"  GPU Temperature: {gpu_temp} °C")

    # Check overall CPU and RAM usage
    print(f"CPU Usage: {psutil.cpu_percent()}%")
    print(f"Memory Usage: {psutil.virtual_memory().percent}%")
    print("-" * 60)

In [ ]:
# Call the function to summarize GPU and system information
print(f"...............Initial - GPU & CPU monitoring...............")
summarize_gpu_info()
print("\n")
# Trigger garbage collection
gc.collect()  # Explicitly call garbage collection to clean up memory

# Setting Up an AlexNet Model for Training in PyTorch

In [ ]:
# Load AlexNet model without pre-trained weights and with output classes
model = models.alexnet(weights=None, num_classes=NUM_CLASSES).to(device) 
# Modify the first convolutional layer to accept grayscale images (1 input channel)
model.features[0] = nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2)
# Move the model to the specified device (GPU/CPU)
model = model.to(device)

### Model Architecture

In [ ]:
print("\n" + "="*60)
print("           Model & Data Pipeline Summary")
print("="*60)
# Print the model architecture for verification
print(f"Model Architecture:\n{model}")

### Model Summary

In [ ]:
# Print a summary of the model
print("\n\nModel Summary:")
summary(model, (1, 224, 224))

# Hyperparameters

In [ ]:
# Hyperparameters
split_ratio = 0.85 # Split training set into 85% for training and 15% for validation
batch_size = 128 # Number of samples per batch; larger sizes speed up training but require more memory
num_epochs = 300 # Number of times to iterate over the entire training dataset
learning_rate = 0.01 # Controls how much the model's weights are updated during training

# Early stopping parameters to prevent overfitting
patience = 15  # Stop training if validation loss doesn't improve for 'patience' epochs
best_val_loss = float('inf')  # Track the best validation loss encountered during training
epochs_without_improvement = 0  # Counter to monitor no improvement in validation loss

# Data loading parameters
num_workers = 16  # Number of subprocesses for data loading

# Define the loss function used to train the model.
criterion = nn.CrossEntropyLoss()
# Define the optimizer used to update model parameters.
# The learning rate controls how large the model's weight updates are during training.
# Momentum helps accelerate gradients vectors in the right directions, thus speeding up convergence.
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001)
# Set up the scheduler
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)  # Example scheduler to reduce learning rate by 0.1 every 30 epochs

# Data Augmentation and Normalization

In [ ]:
# Data augmentation and normalization for MNIST
train_transform = transforms.Compose([
    transforms.Resize(224), # Resize images to 224x224 to fit AlexNet
    transforms.RandomHorizontalFlip(), # Random horizontal flip
    transforms.RandomRotation(20), # Random rotation with a degree range (-20 to 20 degrees)
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=10), # Random affine transformation with translation, scaling, and shear
    transforms.ToTensor(), # Convert PIL image to tensor
    transforms.Normalize(mean=[0.17222732305526733], std=[0.3309466242790222]), # Normalize MNIST images using its mean and standard deviation
])

# Transformations for MNIST test and validation datasets (no augmentation)
val_test_transform = transforms.Compose([
    transforms.Resize(224), # Resize images to 224x224 to fit AlexNet
    transforms.ToTensor(), # Convert PIL image to tensor
    transforms.Normalize(mean=[0.17222732305526733], std=[0.3309466242790222]), # Normalize MNIST images using its mean and standard deviation
])

# Load MNIST
* Split into Training,Testing & Validation sets

In [ ]:
class TransformedDataset(Dataset):
    def __init__(self, dataset, transform):
        # Wrap an existing dataset and apply a transform to each image
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, index):
        # Apply the transform to the image and return it with the label
        img, label = self.dataset[index]
        return self.transform(img), label

    def __len__(self):
        # Return the size of the dataset
        return len(self.dataset)

# Load MNIST dataset 
train_set = datasets.EMNIST(root='./data', split='letters', train=True, download=True, transform=None) # Without any transformations (no augmentation)
test_set = datasets.EMNIST(root='./data', split='letters', train=False, download=True, transform=val_test_transform)  # Test set with transformations

# Split training set into p% for training and 1-p% for validation
train_size = int(split_ratio * len(train_set))  # p% for training
val_size = len(train_set) - train_size   # 1-p% for validation
raw_train, raw_val = random_split(train_set, [train_size, val_size])  # Randomly split the dataset

# Apply the transformations after splitting (for training and validation datasets)
train_dataset = TransformedDataset(raw_train, train_transform)  # Apply train transformations to the training set
val_dataset = TransformedDataset(raw_val, val_test_transform)  # Apply val/test transformations to the validation set

# Create DataLoader for training
train_loader = DataLoader(train_dataset, 
                          batch_size=batch_size,  # Number of samples per batch
                          shuffle=True,           # Shuffle dataset at each epoch
                          num_workers=num_workers, # Number of subprocesses for data loading
                          pin_memory=True)        # Pin memory for faster data transfer to GPU

# Create DataLoader for validation
val_loader = DataLoader(val_dataset, 
                        batch_size=batch_size,  # Same batch size as training
                        shuffle=False,          # No shuffling for validation
                        num_workers=num_workers, # Number of subprocesses for validation
                        pin_memory=True)        # Pin memory for faster data transfer

# Create DataLoader for testing
test_loader = DataLoader(test_set, 
                         batch_size=batch_size,  # Same batch size for testing
                         shuffle=False,          # No shuffling for testing
                         num_workers=num_workers, # Number of subprocesses for testing
                         pin_memory=True)        # Pin memory for faster data transfer

# Check number of images in each DataLoader
print("\n" + "-"*60)
print(f"Number of images in train_loader: {len(train_loader.dataset)}") # Number of images in training dataset
print(f"Number of images in val_loader: {len(val_loader.dataset)}") # Number of images in validation dataset
print(f"Number of images in test_loader: {len(test_loader.dataset)}") # Number of images in testing dataset
print("-" * 60)

# Multi-GPU Training and Early Stopping in PyTorch

In [ ]:
# Check if multiple GPUs are available and use DataParallel for parallel training
# DataParallel allows splitting the input batch across multiple GPUs to speed up training.
# This is useful for larger models or when have access to a multi-GPU setup.
if torch.cuda.device_count() > 1:  # Check if more than one GPU is available
    # Wrap the model with DataParallel to enable multi-GPU training
    model = nn.DataParallel(model)  # This will distribute the input data across available GPUs

In [ ]:
def Equalize_Filters(model):
    if isinstance(model, nn.DataParallel):
        model = model.module
    with torch.no_grad():
        conv1_weights = model.features[0].weight
        conv1_bias = model.features[0].bias
        for i in range(1, ALPHA * 2, 2):
            conv1_weights[i].copy_(conv1_weights[i - 1])
            conv1_bias[i].copy_(conv1_bias[i - 1])

## Train Function

In [ ]:
def train_loop(model, train_loader, optimizer, criterion, device):
    model.train() # Set model to training mode
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    # Taining Loop
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        # Move inputs and labels to the target device (GPU/CPU)
        inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        # Subtract 1 from the target values to shift them to the range 0-25
        labels = (labels - 1).to(device)
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward Pass
        outputs = model(inputs) # Output shape=[batch_size, num_classes]
        loss = criterion(outputs, labels) # Compute Loss

        # Backpropagation and optimization
        loss.backward() # Backpropagation
        optimizer.step() # Update parameters

        # Update running statistics
        running_loss += loss.item() * inputs.size(0) # Aggregate batch Losses per epoch
        predictions = outputs.argmax(dim=1) # Predicted classes
        correct_predictions += (predictions == labels).sum().item() # Actual and Predicted labels
        total_samples += labels.size(0) # Aggregate Total Number of labels
        
        # Batch-wise Information (Loss and Accuracy)
        num_batches = len(train_loader)
        batch_print_interval = max(1, num_batches // 5)
        if batch_idx % batch_print_interval == 0:  # Print logic for batch information
            batch_loss = loss.item()
            batch_accuracy = 100 * correct_predictions / total_samples
            print(f"Batch {batch_idx+1}/{len(train_loader)} - Batch Loss: {batch_loss:.6f}, Batch Accuracy: {batch_accuracy:.4f}%")

    # Calculate Training loss and Accuracy for the epoch
    train_loss = running_loss / total_samples # Compute Average loss per batch
    train_accuracy = 100 * correct_predictions / total_samples # Compute Total accuracy for the epoch

    return train_loss, train_accuracy

## Validation / Test Function

In [ ]:
def val_test_loop(model, data_loader, criterion, device):
    model.eval() # Set model to evaluation mode
    loss = 0.0
    correct_predictions = 0
    total_samples = 0

    # Validation Loop
    with torch.no_grad(): # Detach gradients for Validation
        for inputs, labels in data_loader:
            # Move inputs and labels to the target device (GPU/CPU)
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            # Subtract 1 from the target values to shift them to the range 0-25
            labels = (labels - 1).to(device)
            # Forward Pass
            outputs = model(inputs) # Output shape=[batch_size, num_classes]
            loss += criterion(outputs, labels).item() * inputs.size(0) # Compute Loss and Aggregate
    
            # Update running statistics
            predictions = outputs.argmax(dim=1) # Predicted classes
            correct_predictions += (predictions == labels).sum().item()
            total_samples += labels.size(0) # Aggregate Total Number of labels

    # Calculate Average Validation loss and Accuracy for the epoch
    total_loss = loss / total_samples  # Compute Average loss per batch
    total_accuracy = 100 * correct_predictions / total_samples # Compute Total accuracy for the epoch

    return total_loss, total_accuracy

# Saving AlexNet Metrics on ImageNet
### Save Epoch Results 
* In a .csv and append data after each epoch

In [ ]:
def save_epoch_results(epoch, current_lr, running_train_loss, running_train_accuracy, val_loss, val_accuracy, KE_val_loss, KE_val_accuracy,
                        epoch_time, KE_time, KE_with_V_time, FOLDER_NAME):
    # Prepare the data for the current epoch
    epoch_data = {
        'Epoch': [epoch+1],
        'LR' : [current_lr],
        # Losses & Accuracies Before Kernel Equalization
        'Running Train Loss': [running_train_loss], 
        'Running Train Accuracy': [running_train_accuracy],
        'Validation Loss': [val_loss],
        'Validation Accuracy': [val_accuracy],
        # Losses & Accuracies AFter Kernel Equalization
        'KE Validation Loss': [KE_val_loss],
        'KE Validation Accuracy': [KE_val_accuracy],
        # Time consumptions
        'Epoch Time': [epoch_time],
        'KE Time': [KE_time],
        'KE + Forward Pass Time': [KE_with_V_time],
    }

    # Convert to DataFrame
    df = pd.DataFrame(epoch_data)

    # Define the file path where the results will be saved
    file_path = os.path.join(FOLDER_NAME, f'{FOLDER_NAME}-a{ALPHA}-epoch_results.csv')
    
    # Check if the file exists, if not include header while saving
    header = not os.path.exists(file_path)
    
    # Now, save+append the results to the CSV
    df.to_csv(file_path, mode='a', header=header, index=False)

    print(f"Epoch {epoch} results saved...")

## Checkpoint Function

In [ ]:
def save_checkpoint(model, optimizer, epoch, FOLDER_NAME):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    # Define the checkpoint path (includes folder and filename)
    checkpoint_path = os.path.join(FOLDER_NAME, f"{FOLDER_NAME}-a{ALPHA}-checkpoint_epoch_{epoch+1}.pth")
    # Save the checkpoint
    torch.save(checkpoint, checkpoint_path)
    print(f"Checkpoint saved at epoch {epoch+1} to {checkpoint_path}...")

# Training with Early Stopping

In [ ]:
print("\n" + "#" * 60)
print(f"#         TRAINING STARTED: {MODEL} on {DATASET} Dataset        #")
print("#" * 60 + "\n")

start_time = time.time()

for epoch in range(num_epochs):
    epoch_start = time.time()

    print("\n" + "="*60)
    print(f"........... Epoch {epoch+1} Start - GPU & CPU monitoring ...........")
    gc.collect()
    torch.cuda.empty_cache()
    summarize_gpu_info()

    print("\n" + "="*60)
    print(f"...................... Epoch {epoch+1} Start .......................")
    print("="*60)

    running_train_loss, running_train_accuracy = train_loop(model, train_loader, optimizer, criterion, device)
    val_loss, val_accuracy = val_test_loop(model, val_loader, criterion, device)
    epoch_time = time.time() - epoch_start 
    
    KE_start = time.time()
    Equalize_Filters(model)
    KE_time = time.time() - KE_start
    
    KE_val_loss, KE_val_accuracy = val_test_loop(model, val_loader, criterion, device)
    KE_with_V_time = time.time() - KE_start
    
    print("\n" + "="*60)
    print(f"...................... Epoch {epoch+1} Results .....................")
    print('--------------- Before Kernel Equalization -----------------')
    print(f"Epoch {epoch+1}/{num_epochs} Summary:")
    print("-" * 60)
    print(f"Running Train Loss: {running_train_loss:.6f}, Running Train Accuracy: {running_train_accuracy:.4f}%")
    print(f"Validation Loss: {val_loss:.6f}, Validation Accuracy: {val_accuracy:.4f}%")

    print('\n---------------- After Kernel Equalization -----------------')
    print(f"Epoch {epoch+1}/{num_epochs} Summary:")
    print("-" * 60)
    print(f"Validation Loss: {KE_val_loss:.6f}, Validation Accuracy: {KE_val_accuracy:.4f}%")
    
    print(f"\n~~~*~~~ Time Consumptions ~~~*~~~")
    print(f"Epoch {epoch+1} completed in {epoch_time/60:.4f} minutes.")
    print(f"Time Consumption of Kernel Equalization: {KE_time:.6f} seconds.")
    print(f"Time Consumption of Kernel Equalization + Forward Pass Validation dataset: {KE_with_V_time/60:.4f} minutes.")
    print(f"Time Elapsed Since Epoch {epoch + 1} Started: {(time.time() - epoch_start)/60:.4f} minutes.")
    print(f"Total Time Elapsed Since Training Started: {(time.time() - start_time)/60:.4f} minutes.\n")  
        
    current_lr = scheduler.get_last_lr()[0]

    save_epoch_results(epoch, current_lr, running_train_loss, running_train_accuracy, val_loss, val_accuracy, KE_val_loss, KE_val_accuracy,
                        epoch_time, KE_time, KE_with_V_time, FOLDER_NAME)

    print(f"\n.........Epoch {epoch+1} End - GPU & CPU monitoring.........")
    summarize_gpu_info()

    if (epoch + 1) % 30 == 0:
        save_checkpoint(model, optimizer, epoch, FOLDER_NAME)

    print("="*60)
    print("\n")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
        model_weights_path = os.path.join(FOLDER_NAME, f"{FOLDER_NAME}-a{ALPHA}-best_model.pth")
        torch.save(model.state_dict(), model_weights_path)
    else:
        epochs_without_improvement += 1
        
    if epochs_without_improvement >= patience:
        print("\n================ Early Stopping Triggered! =================\n")
        break

    scheduler.step()
    torch.cuda.empty_cache()
    gc.collect()

end_time = time.time()
total_time = end_time - start_time

print(f"\nTraining complete! Total time: {total_time / 60:.4f} minutes.")
print(f"                             : {total_time / 3600:.4f} hours.")
print(f"                             : {total_time / 86400:.4f} days.")
print("=" * 60)
print("\n*~*~*~*~*~*~*~*~*~*~*~*~* THE END *~*~*~*~*~*~*~*~*~*~*~*~*~*\n")
print("=" * 60)

nvmlShutdown()

# Load the Model Structure and assign Best Model

In [ ]:
# CoupledNet Model
Best_model = models.alexnet(weights=None, num_classes=NUM_CLASSES) # Load AlexNet model without pre-trained weights and with output classes
Best_model.features[0] = nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2)
Best_model = Best_model.to(device)

if torch.cuda.device_count() > 1:
    Best_model = nn.DataParallel(Best_model) # Only wrap DDP if multi-GPU
    
# Load the model and weights
weights_path = os.path.join(FOLDER_NAME, f"{FOLDER_NAME}-a{ALPHA}-best_model.pth")  # Adjust this path to the correct file
state_dict = torch.load(weights_path, weights_only=True)

new_state_dict = {}
if any(key.startswith("module.") for key in state_dict.keys()):
    # All keys have module. prefix
    new_state_dict = state_dict
else:
    # Add module. prefix if missing (needed for DataParallel)
    for key, value in state_dict.items():
        new_state_dict["module." + key] = value

Best_model.load_state_dict(new_state_dict)

# Equalize filters
Equalize_Filters(Best_model)

# Reverse Engineering model
RE_model = models.alexnet(weights=None, num_classes=NUM_CLASSES)
RE_model.features[0] = nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2)
RE_model = RE_model.to(device)

if torch.cuda.device_count() > 1:
    RE_model = nn.DataParallel(RE_model)

# top-1 Accuracy & top-5 Accuracy

In [ ]:
def top_accuracy(model, data_loader, criterion, device):
    model.eval()  # Set model to evaluation mode
    loss = 0.0
    correct_predictions_top1 = 0
    correct_predictions_top5 = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            # Move inputs and labels to the target device (GPU/CPU)
            inputs, labels = inputs.to(device), labels.to(device)
            # Subtract 1 from the target values to shift them to the range 0-25
            labels = (labels - 1).to(device)
            # Forward Pass
            outputs = model(inputs)
            loss += criterion(outputs, labels).item()

            # Get top-5 predictions for each sample
            _, top5_predictions = outputs.topk(5, dim=1)

            # Ensure both top5_predictions and labels are on the same device
            top5_predictions = top5_predictions.to(device)  # Move top5 predictions to the same device as labels

            # Check for Top-1 accuracy
            top1_predictions = outputs.argmax(dim=1)
            correct_predictions_top1 += (top1_predictions == labels).sum().item()

            # Check for Top-5 accuracy (true label should be in the top 5 predictions)
            correct_predictions_top5 += (top5_predictions == labels.view(-1, 1)).any(dim=1).sum().item()

            total_samples += labels.size(0)

    # Calculate Average Validation loss and Accuracy for the epoch
    avg_loss = loss / len(data_loader)
    top1_accuracy = 100 * correct_predictions_top1 / total_samples
    top5_accuracy = 100 * correct_predictions_top5 / total_samples

    return avg_loss, top1_accuracy, top5_accuracy

# Inferencing on Validation Dataset

## CoupledNet Validation Accuracy

In [ ]:
# Set the model to evaluation mode
Best_model.eval()

avg_loss, top1_accuracy, top5_accuracy = top_accuracy(Best_model, val_loader, criterion, device)

# Print the final Validation loss and accuracy
print('\n')
print("\n" + "="*60)
print("######################## CoupledNet ########################")
print("------------- Inference on Validation Dataset --------------")
print(f"Top-1 Accuracy: {top1_accuracy:.4f}%, Top-5 Accuracy: {top5_accuracy:.4f}%, Validation Loss: {avg_loss:.6f}.")
print("="*60)

In [ ]:
# Close the file
log_file.close()

# Reset stdout to default
sys.stdout = sys.__stdout__